In [9]:
from NYT_api_advanced import wraper_function_data, handle_multiple_words, \
                            produce_wordclouds, save_images_as_str, \
                            plotly_url, writing_js_file, writing_html_file

## GET request to NY Times API

In [11]:
#define the terms of your query to the NY Times API
terms = 'Neural Nets'
year_start = 2006
year_end = 2016

#credential to send GET request to API
path_to_cred = '../../credentials/credentials.yml'

In [12]:
# getting the data from NYT API
dict_hits, d_keywords = wraper_function_data(year_start, year_end, terms, path_to_cred)

## Wordclouds

I used **https://github.com/amueller/word_cloud** to generate the wordclouds with standard settings (a mix of rank importance and frequency sets the size of the word in the cloud)

You need to install Wordclouds (see the GitHub repository)

The words given must be in a string, they are then separated into individual words. For instance, NYT keywords *'Clinton, Bill'* will give *'Clinton'* and *'Bill'* ranked and counted separately. Since I wanted to see the keyword as a whole, I linked then with *'_'*, and therefore the wordclouds will have *Clinton_Bill*.

In [13]:
#transform 'Clinton, Bill' into 'Clinton_Bill'
d_linked_keywords = handle_multiple_words(d_keywords)

#make the wordclouds (as matplotlib figures)
dict_figs = produce_wordclouds(d_linked_keywords, plot_option=False)

## Interactive Bar Graph with image on hover

I choose to got with plotly graphs (https://plot.ly/) as there was a great example of what I wanted to do on GitHub (https://github.com/etpinard/plotly-dashboards/tree/master/hover-images). This means I need to update the *index.html* and the *main.js* to have my images and plot.

#### $\longrightarrow$ Writing main.js
When exploring the Javascript file, we can see that a Javascript dictionary allows us to link the points (passed as a list in `x` in the plotly generator code) and the hover image, in the type 'point': 'image'. Interestingly, the image is passed as a text, so our wordclouds have to be encoded into text 

limitation: I was not able to use intergers or integers as strings as the key. The key will have '-' instead of whitespaces and all text is lowered.

#### $\longrightarrow$ Writing index.htlm
- a plotly url needs to be included in the html, I needed to generate the interactive bar graph (no hover for now). You can register for free on plotly, and you will need to create a credential file.
>plotly.tools.set_credentials_file(username='****', api_key='****')

- a javascript file needs to be written (main.js)

- generating the html

In [14]:
# getting the plotly url, making the bar chart
url = plotly_url(year_start, year_end, dict_hits)

In [15]:
#main.js
dict_str = save_images_as_str(dict_figs)
writing_js_file(dict_str, 'main_explore.js')

"(function main() {\n\n    var Plot = {\n        id: 'plot',\n        imgId: 'hover-image',\n        domain: 'https://plot.ly'\n    };\n\n    Plot.iframe = document.getElementById(Plot.id);\n    Plot.graphContentWindow = Plot.iframe.contentWindow;\n\n    Plot.hoverImg = document.getElementById(Plot.imgId);\n\n    Plot.init = function init() {\n        var pinger = setInterval(function() {\n            Plot.post({task: 'ping'});\n        }, 500);\n\n        function messageListener(e) {\n            var message = e.data;\n\n            if(message.pong) {\n                console.log('Initial pong, frame is ready to receive');\n                clearInterval(pinger);\n\n                Plot.post({\n                    'task': 'listen',\n                    'events': ['hover']\n                });\n            }\n            else if(message.type === 'hover') {\n                Plot.onHover(message);\n            }\n        }\n\n        window.removeEventListener('message', messageListener)

In [16]:
#html
writing_html_file(url, 'main_explore.js', 'index.html')

'\n    <!DOCTYPE html>>\n    <style>\n    /* http://stackoverflow.com/questions/11555809/image-overhead-iframe */\n    iframe{\n      width: 1000px;\n      height: 600px;\n      border : none;\n      z-index: 2;\n    }\n\n    img{\n      position: absolute;\n      left: 400px;\n      top: 130px;\n      z-index: 3;\n      border : none;\n      max-width: 400px;\n      max-height: 300px;\n      align: "middle";\n    }\n    </style>\n    <body>\n\n    <iframe id="plot" src="https://plot.ly/~AnnaVM/37" seamless></iframe>\n\n    <!-- http://stackoverflow.com/questions/11555809/image-overhead-iframe -->\n    \n\n    <script src=\'main_explore.js\'></script>\n\n    </body>\n\n    '

## Exploring the interactive graph

in order to have the js execute, run the html on a local server with the command in terminal 

> $python -m SimpleHTTPServer

the file protocol does not execute the Javascript properly